# Noiseless QCNN (8-qubits) demo for Model 2

This demo uses 8-qubit Quantum Convolutional Neural Network (QCNN) to see how pre-training the quantum embedding can be helpful for training a parameterized QML circuits for classfication tasks.

If you are interested in the details about the QCNN used in this demo, check out https://arxiv.org/pdf/2108.00661.pdf.

In [ ]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight

import Hybrid_nn
import torch
from torch import nn
import data_GBA_1_1
import pennylane as qml
import embedding

## 0. Getting Started

Load the dataset with four features

In [ ]:
dev = qml.device('default.qubit', wires=8)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data_GBA_1_1.data_load_and_process('protein', feature_reduction, classes)
print("X_train:",X_train.shape,"/X_test:",X_test.shape,"/Y_train:",Y_train.shape,"/Y_test:",Y_test.shape)

def new_data(batch_size, X, Y):
    X1_new, X2_new, Y_new = [], [], []
    for i in range(batch_size):
        n, m = np.random.randint(len(X)), np.random.randint(len(X))
        X1_new.append(X[n])
        X2_new.append(X[m])
        if Y[n] == Y[m]:
            Y_new.append(1)
        else:
            Y_new.append(0)
    return torch.tensor(X1_new).to(device), torch.tensor(X2_new).to(device), torch.tensor(Y_new).to(device)

def new_data_balanced(batch_size, X, Y):
    X1_new, X2_new, Y_new = [], [], []
    class_0_indices = np.where(Y == 0)[0]
    class_1_indices = np.where(Y == 1)[0]
    
    half_batch = batch_size // 2
    
    for _ in range(half_batch):
        n = np.random.choice(class_0_indices)
        m = np.random.choice(class_1_indices)
        X1_new.append(X[n])
        X2_new.append(X[m])
        Y_new.append(0)

    for _ in range(half_batch):
        n = np.random.choice(class_1_indices)
        m = np.random.choice(class_1_indices)
        while n == m: 
            m = np.random.choice(class_1_indices)
        X1_new.append(X[n])
        X2_new.append(X[m])
        Y_new.append(1)


    X1_new, X2_new, Y_new = np.array(X1_new), np.array(X2_new), np.array(Y_new)
    X1_new, X2_new, Y_new = torch.tensor(X1_new).to(torch.float32), torch.tensor(X2_new).to(torch.float32), torch.tensor(Y_new).to(torch.float32)
    
    if not feature_reduction:
        #X1_new = X1_new.permute(0, 3, 1, 2)
        #X2_new = X2_new.permute(0, 3, 1, 2)
        X1_new = X1_new
        X2_new = X2_new
    
    return X1_new.to(device), X2_new.to(device), Y_new.to(device)

N_valid, N_test = 33, 33
X1_new_valid, X2_new_valid, Y_new_valid = new_data_balanced(N_valid, X_test, Y_test)
X1_new_test, X2_new_test, Y_new_test = new_data_balanced(N_test, X_test, Y_test)

In [ ]:
print("Y_train, Y_test:", Counter(Y_train), Counter(Y_test))
Y_new_valid_check = [a.tolist() for a in Y_new_valid]
Y_new_test_check = [a.tolist() for a in Y_new_test]
print("Y_new_valid, Y_new_test:", Counter(Y_new_valid_check), Counter(Y_new_test_check))

# Part1: Pre-Training the Embedding

Circuit for evaluating Model2_Fidelity and Model2_HSinner for 8 qubits

In [ ]:
@qml.qnode(dev, interface="torch")
def circuit2(inputs):
    #print("inputs:", inputs.shape)
    return embedding.XYZ_embedding_with_inverse(input1=inputs[:90], input2=inputs[90:])

qlayer2 = qml.qnn.TorchLayer(circuit2, weight_shapes={})

def new_qlayer2(inputs):
    outputs=[]
    for i in range(inputs.shape[0]):
        output = qlayer2(inputs[i,:])
        outputs.append(output)
    stacked = torch.stack(outputs)
    return stacked

class Model2_Fidelity(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(39, 1240),
            nn.ReLU(),
            nn.Linear(1240, 620),
            nn.ReLU(),
            nn.Linear(620, 360),
            nn.ReLU(),
            nn.Linear(360, 180),
            nn.ReLU(),
            nn.Linear(180, 90)    
        )

    def forward(self, x1, x2):
        x1 = self.linear_relu_stack2(x1)
        x2 = self.linear_relu_stack2(x2)
        #print("x1:",x1.shape,"/x2:",x2.shape)

        x = torch.concat([x1, x2], 1)
        #print("x:",x.shape)
        x = new_qlayer2(x)
        return x[:,0]

Calculate the distances of Test dataset with the pre-trained quantum embeddings. From the calculated trace distance gain the lower bound of the linear loss function (with respect to the test data).

In [ ]:
X1_test, X0_test = [], []
for i in range(len(X_test)):
    if Y_test[i] == 1:
        X1_test.append(X_test[i])
    else:
        X0_test.append(X_test[i])
        
X1_test, X0_test = np.array(X1_test), np.array(X0_test)
X1_test, X0_test = torch.tensor(X1_test), torch.tensor(X0_test)

X1_train, X0_train = [], []
for i in range(len(X_train)):
    if Y_train[i] == 1:
        X1_train.append(X_train[i])
    else:
        X0_train.append(X_train[i])

X1_train, X0_train = np.array(X1_train), np.array(X0_train)
X1_train, X0_train = torch.tensor(X1_train), torch.tensor(X0_train)

In [ ]:
@qml.qnode(dev, interface="torch")
def Four_Distance2(inputs): 
    embedding.XYZ_embedding(inputs[0:90])
    return qml.density_matrix(wires=range(8)) 

Distance2_qlayer2 = qml.qnn.TorchLayer(Four_Distance2, weight_shapes={})

def Distance_qlayer2(inputs):
    outputs=[]
    for i in range(inputs.shape[0]):
        output = Distance2_qlayer2(inputs[i,:])
        outputs.append(output)
    stacked = torch.stack(outputs)
    return stacked

class Distances2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer2_distance = qml.qnn.TorchLayer(Four_Distance2, weight_shapes={})
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(39, 1240),
            nn.ReLU(),
            nn.Linear(1240, 620),
            nn.ReLU(),
            nn.Linear(620, 360),
            nn.ReLU(),
            nn.Linear(360, 180),
            nn.ReLU(),
            nn.Linear(180, 90)
        )
    def forward(self, x1, x0, Distance):
        x1 = self.linear_relu_stack2(x1.float())
        x0 = self.linear_relu_stack2(x0.float())
        rhos1 = Distance_qlayer2(x1)
        rhos0 = Distance_qlayer2(x0)
        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0
        if Distance == 'Trace':
            eigvals = torch.linalg.eigvals(rho_diff)
            return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))
        elif Distance == 'Hilbert-Schmidt':
            return 0.5 * torch.trace(rho_diff @ rho_diff)


Model2_Fidelity_Distance = Distances2().to(device)

Trace_before_traindata = Model2_Fidelity_Distance(X1_train, X0_train, 'Trace')
Trace_before_testdata = Model2_Fidelity_Distance(X1_test, X0_test, 'Trace')
print(f"Trace Distance (Training Data) Before: {Trace_before_traindata}")
print(f"Trace Distance (Test Data) Before: {Trace_before_testdata}")

LB_before_traindata = 0.5 * (1 - Trace_before_traindata)

PATH_Model2_Fidelity = '/Users/jungguchoi/Library/Mobile Documents/com~apple~CloudDocs/1_Post_doc(Cleveland_clinic:2024.10~2025.09)/1_Research_project/3_quantum_embedding_comparison_sequence(2024.09 ~ XXXX.XX)/2_exp/59_Dr_Park_Comments_SEP0925/1_LIT-PCBA_new_ansatz/0_Neural-Quantum-Embedding-main-cond1-NQE_XY/Results/Model2_Fidelity_Protein_8_qubits2.pt'
Model2_Fidelity_Distance.load_state_dict(torch.load(PATH_Model2_Fidelity, map_location=device), strict=False)

# Distances After training with Model2_Fidelity
Trace_Fidelity_traindata = Model2_Fidelity_Distance(X1_train, X0_train, 'Trace')
Trace_Fidelity_testdata = Model2_Fidelity_Distance(X1_test, X0_test, 'Trace')
print(f"Trace Distance (Training Data) After Model2 Fidelity: {Trace_Fidelity_traindata}")
print(f"Trace Distance (Test Data) After Model2 Fidelity: {Trace_Fidelity_testdata}")

# Lower Bounds
LB_Fidelity_traindata = 0.5 * (1 - Trace_Fidelity_traindata.detach().numpy())

# Part2: Training QCNN with/without Pre-trained embedding

In [ ]:
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(39, 360),
            nn.ReLU(),
            nn.Linear(360, 180),
            nn.ReLU(),
            nn.Linear(180, 90)
        )
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()

model = x_transform2().to(device)

Tunable Hyperparameters

In [ ]:
steps = 2000
learning_rate = 0.001
batch_size = 25
ansatz = 'SU4'

In [ ]:
Y_total = np.array(Y_train+Y_test)
print(len(Y_total))
print(np.unique(Y_total))

y_transformed = np.where(Y_total == -1, 0, 1)

In [ ]:
class_weight = compute_class_weight(class_weight="balanced", classes=np.unique(y_transformed), y=y_transformed)
print("Class weight:", class_weight)
class_weight_dict = {-1: class_weight[0], 1: class_weight[1]}
print("Fixed class weight:", class_weight_dict)

In [ ]:
def statepreparation(x, Trained):
    if Trained == 'Model2_Fidelity':
        model.load_state_dict(torch.load(PATH_Model2_Fidelity, map_location=device))
        x = model(torch.tensor(x).float())
    elif Trained == 'Model2_HSinner':
        model.load_state_dict(torch.load(PATH_Model2_HSinner, map_location=device))
        x = model(torch.tensor(x))
    embedding.XYZ_embedding(x)

@qml.qnode(dev)
def QCNN_classifier(params, x, Trained):
    statepreparation(x, Trained)
    embedding.QCNN_eight(params, ansatz)
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)

def Linear_Loss_class_weights(labels, predictions, class_weight):
    loss = 0
    for l,p in zip(labels, predictions):
        weight = class_weight[int(l)]
        loss += weight * 0.5 * (1 - l * p)
    return loss / len(labels)

def cost(weights, X_batch, Y_batch, Trained):
    preds = [QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)

def cost_class_weights(weights, X_batch, Y_batch, Trained, class_weight):
    preds = [QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss_class_weights(Y_batch, preds, class_weight)


def circuit_training(X_train, Y_train, Trained, class_weight):

    if ansatz == 'SU4':
        num_weights = 45
    elif ansatz == 'U_6':
        num_weights = 20
    elif ansatz == 'TTN':
        num_weights = 4

    weights = np.random.random(num_weights, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost_class_weights(v, X_batch, Y_batch, Trained, class_weight),
                                                     weights)
        loss_history.append(cost_new)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

In [ ]:
Loss_histories_not_trained, weights_not_trained = [], []
Loss_histories_Model2_Fidelity, weights_Model2_Fidelity = [], []
for i in range(5):
    loss_Model2_Fidelity, weight_Model2_Fidelity = circuit_training(X_train, Y_train, 'Model2_Fidelity', class_weight_dict)
    loss_not_trained, weight_not_trained = circuit_training(X_train, Y_train, False, class_weight_dict)
    
    Loss_histories_not_trained.append(loss_not_trained)
    weights_not_trained.append(weight_not_trained)

    Loss_histories_Model2_Fidelity.append(loss_Model2_Fidelity)
    weights_Model2_Fidelity.append(weight_Model2_Fidelity)

Loss_histories_not_trained = np.array(Loss_histories_not_trained)
Loss_histories_Model2_Fidelity = np.array(Loss_histories_Model2_Fidelity) 
#Loss_histories_Model2_HSinner = np.array(Loss_histories_Model2_HSinner)

Not_trained_mean, Not_trained_std = Loss_histories_not_trained.mean(axis=0), Loss_histories_not_trained.std(axis=0)
Model2_Fidelity_mean, Model2_Fidelity_std = Loss_histories_Model2_Fidelity.mean(axis=0), Loss_histories_Model2_Fidelity.std(axis=0)

In [ ]:
print(len(Loss_histories_not_trained), len(weights_not_trained))
print(len(Loss_histories_Model2_Fidelity), len(weights_Model2_Fidelity))

In [ ]:
Loss_histories_not_trained = np.array(Loss_histories_not_trained)
Loss_histories_Model2_Fidelity = np.array(Loss_histories_Model2_Fidelity) 

Not_trained_mean ,Not_trained_std = Loss_histories_not_trained.mean(axis=0), Loss_histories_not_trained.std(axis=0)
Model2_Fidelity_mean, Model2_Fidelity_std = Loss_histories_Model2_Fidelity.mean(axis=0), Loss_histories_Model2_Fidelity.std(axis=0)

In [ ]:
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 5]
fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 3)
with sns.axes_style("darkgrid"):
    ax.plot(range(len(Not_trained_mean)), Not_trained_mean, label="No Pre-training", c=clrs[0])
    ax.fill_between(range(len(Not_trained_mean)), Not_trained_mean-Not_trained_std, Not_trained_mean+Not_trained_std, alpha=0.3,facecolor=clrs[0])

    ax.plot(range(len(Model2_Fidelity_mean)), Model2_Fidelity_mean, label="Model2 Fidelity Pre-training", c=clrs[1])
    ax.fill_between(range(len(Model2_Fidelity_mean)), Model2_Fidelity_mean-Model2_Fidelity_std, Model2_Fidelity_mean+Model2_Fidelity_std, alpha=0.3,facecolor=clrs[1])

    #ax.plot(range(len(Model2_HSinner_mean)), Model2_HSinner_mean, label="Model2 HSinner Pre-training", c=clrs[2])
    #ax.fill_between(range(len(Model2_HSinner_mean)), Model2_HSinner_mean-Model2_HSinner_std, Model2_HSinner_mean+Model2_HSinner_std, alpha=0.3,facecolor=clrs[2])

    ax.plot(range(2000), np.ones(2000) * LB_before_traindata.item(), linestyle='dashed', linewidth=1.5, label="Lower Bound without Pre-training", c=clrs[0])
    ax.plot(range(2000), np.ones(2000) * LB_Fidelity_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Model2 Fidelity", c=clrs[1])
    #ax.plot(range(1000), np.ones(1000) * LB_HSinner_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Model2 HSinner", c=clrs[2])


ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.set_title("PCA+NQE+QCNN (8qubits) Loss History")
ax.legend()

Save the Loss Histories and Trained weights

In [ ]:
save_dir = '/Users/jungguchoi/Library/Mobile Documents/com~apple~CloudDocs/1_Post_doc(Cleveland_clinic:2024.10~2025.09)/1_Research_project/3_quantum_embedding_comparison_sequence(2024.09 ~ XXXX.XX)/2_exp/9_NQE_QCNN_MNIST_8_qubits/Neural-Quantum-Embedding-main-8-qubits/Results/4_241021_8_qubits_cond2_PCA_NQE_QCNN(from_177_to_16_in_NN)/'
f = open(save_dir+'/Loss_histories_and_weights.txt', 'w')

for i in range(5):
    f.write(f'Loss History Model2 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_Model2_Fidelity[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights Model2 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(weights_Model2_Fidelity[i]))

#for i in range(5):
#    f.write(f'Loss History Model2 HSinner {i + 1}:')
#    f.write('\n')
#    f.write(str(Loss_histories_Model2_HSinner[i]))
#    f.write('\n')
#for i in range(5):
#    f.write(f'Weights Model2 HSinner {i + 1}:')
#    f.write('\n')
#    f.write(str(weights_Model2_HSinner[i]))
f.close()

Check the accuracies of QCNN classifiers

In [ ]:
def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)


accuracies_not_trained = []
accuracies_Model2_Fidelity, accuracies_Model2_HSinner = [], []

for i in range(4):
    prediction_not_trained = [QCNN_classifier(weights_not_trained[i], x, Trained=False) for x in X_test]
    prediction_Model2_Fidelity = [QCNN_classifier(weights_Model2_Fidelity[i], x, Trained='Model2_Fidelity') for x in X_test]
    #prediction_Model2_HSinner = [QCNN_classifier(weights_Model2_HSinner[i], x, Trained='Model2_HSinner') for x in X_test]
    
    accuracy_not_trained = accuracy_test(prediction_not_trained, Y_test)
    accuracy_Model2_Fidelity = accuracy_test(prediction_Model2_Fidelity, Y_test)
    #accuracy_Model2_HSinner = accuracy_test(prediction_Model2_HSinner, Y_test)

    accuracies_not_trained.append(accuracy_not_trained)
    accuracies_Model2_Fidelity.append(accuracy_Model2_Fidelity)
    #accuracies_Model2_HSinner.append(accuracy_Model2_HSinner)

accuracies_not_trained = np.array(accuracies_not_trained)
accuracies_Model2_Fidelity = np.array(accuracies_Model2_Fidelity)
#accuracies_Model2_HSinner = np.array(accuracies_Model2_HSinner)

print(f" Accuracy without pre-training: {accuracies_not_trained.mean()} ± {accuracies_not_trained.std()}")
print(f" Accuracy after pre-training with Model2_Fidelity: {accuracies_Model2_Fidelity.mean()} ± {accuracies_Model2_Fidelity.std()}")
#print(f" Accuracy after pre-training with Model2_HSinner: {accuracies_Model2_HSinner.mean()} ± {accuracies_Model2_HSinner.std()}")